In [2]:
pip install "unstructured[local-inference]"

Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade langchain

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.9/472.9 KB 750.4 kB/s eta 0:00:00m eta 0:00:010:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.55
    Uninstalling langchain-0.0.55:
      Successfully uninstalled langchain-0.0.55
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
print(sys.path)

['/home/morphine/Desktop/lang', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/morphine/.local/lib/python3.10/site-packages', '/home/morphine/aries-cloudagent-python', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


In [3]:
sys.path.append('<path_to_langchain_installation>')

In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
#Load your data

In [2]:
loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")
#loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")

In [ ]:
# pip uninstall "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"

In [3]:
data = loader.load()

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


In [5]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 201014 characters in your document


In [7]:
#Chunk your data up into smaller documents

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [7]:
print (f'Now you have {len(texts)} documents')

Now you have 240 documents


In [11]:
#Create embeddings of your documents to get ready for semantic search

In [14]:
pip install pinecone-client

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 KB 183.6 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 759.9 kB/s eta 0:00:00MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from tqdm.autonotebook import tqdm

In [2]:
pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 KB 1.7 MB/s eta 0:00:003.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.2/198.2 KB 1.9 MB/s eta 0:00:000:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
OPENAI_API_KEY = '...'
PINECONE_API_KEY = '...'
PINECONE_API_ENV = 'us-east4-gcp'

In [11]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [12]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain"

In [13]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [14]:
query = "What are examples of good data science teams?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [15]:
#Query those docs to get your answer back

In [16]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [17]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [20]:
query = "Could you please explain me about model validation?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [21]:
chain.run(input_documents=docs, question=query)

' Model validation is the process of assessing how well a model fits the observed data. It is used to answer the question “How well did my hypothesis fit the observed data?” To validate a model, the data is usually split into training, testing and validation sets. The model is constructed with the training data and then evaluated with the testing data. The performance of the model against the testing set is used to further reduce model error. Finally, the model is evaluated on the validation data to assess how well the model generalizes. Statistical methods such as calculating the coefficient of determination, commonly called the R-squared value, are also used to validate models.'